In [1]:
import eli5
from eli5.lime import TextExplainer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# from moseq2_nlp.data import get_embedding_representation, get_transition_representation, get_usage_representation, load_groups
import moseq2_nlp.train as trainer
from typing import Literal

from moseq2_viz.util import parse_index
from moseq2_viz.model.util import (get_syllable_statistics,get_transition_matrix, parse_model_results)
from tqdm import tqdm
from typing import Dict, List


# Usages

In [2]:
def load_groups(index_file: str, custom_groupings: List[str]) -> Dict[str, str]:
    # Get group names available in model
    _, sorted_index = parse_index(index_file)
    available_groups = list(set([sorted_index['files'][uuid]['group'] for uuid in sorted_index['files'].keys()]))

    # { subgroup: supergroup }
    group_mapping: Dict[str, str] = {}

    if custom_groupings is None or len(custom_groupings) <= 0:
        for g in available_groups:
            group_mapping[g] = g

    else:
        for supergroup in custom_groupings:
            subgroups = supergroup.split(',')
            for subg in subgroups:
                if subg not in available_groups:
                    print(f'WARNING: subgroup "{subg}" from supergroup "{supergroup}" not found in model! Omitting...')
                    continue

                if subg in group_mapping:
                    print(f'WARNING: subgroup "{subg}" from supergroup "{supergroup}" already registered to supergroup "{group_mapping[subg]}"! Omitting...')
                    continue

                group_mapping[subg] = supergroup

    return group_mapping

In [3]:
def get_usage_representation(model_file: str, index_file: str, group_map: Dict[str, str], max_syllable: int=100):
    _, sorted_index = parse_index(index_file)
    model = parse_model_results(model_file, sort_labels_by_usage=True, count='usage')
    label_group = [sorted_index['files'][uuid]['group'] for uuid in model['keys']]

    usage_vals = []
    out_groups = []
    for l, g in zip(tqdm(model['labels']), label_group):
        if g in group_map.keys():
            u, _ = get_syllable_statistics(l, max_syllable=max_syllable, count='usage')
            u_vals = list(u.values())
            total_u = np.sum(u_vals)
            usage_vals.append(np.array(u_vals) / total_u)
            out_groups.append(group_map[g])

    return out_groups, np.array(usage_vals)

In [4]:
custom_groupings = []
model_file = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/rOT_model_1000.p'
index_path = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/moseq2-index.role.yaml'
group_map = load_groups(index_path, custom_groupings)

num_syllables = 70

out_group, usages = get_usage_representation(model_file, index_path, group_map, num_syllables)


100%|██████████| 80/80 [00:00<00:00, 105.86it/s]


In [5]:
usages.shape

(80, 70)

## rOT

In [6]:
## Get features and labels
features = pd.read_csv('/Users/aishwaryaharpale/tischfield-lab/moseq2_nlp_usages/rOT/features.csv').to_numpy()
labels = pd.read_csv('/Users/aishwaryaharpale/tischfield-lab/moseq2_nlp_usages/rOT/labels.csv').to_numpy()

In [7]:
## Parameters

K = 1
scoring = 'accuracy'
penalty = 'l2'
num_c = 11
seed = 0

In [8]:
## Pipeline classifier
best_C, best_score, clf = trainer.train_regressor(features, labels, K, scoring, penalty, num_c, seed)

/Users/aishwaryaharpale/opt/anaconda3/envs/moseq2-nlp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [9]:
## Raw ELI5
eli5.show_weights(clf)

In [8]:
## Get group map for the model
custom_groupings = []
model_file = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/rOT_model_1000.p'
index_path = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/moseq2-index.role.yaml'
group_map = load_groups(index_path, custom_groupings)

num_transitions = 300
num_syllables = 70

feature_names = get_feature_names(index_path,model_file, group_map)

100%|██████████| 80/80 [00:00<00:00, 80.53it/s]


In [9]:
## ELI5 with transitions names
eli5.show_weights(clf, feature_names=feature_names)

## rST

In [10]:
## Get features and labels
features = pd.read_csv('/Users/aishwaryaharpale/tischfield-lab/moseq2_nlp_usages/rST/features.csv').to_numpy()
labels = pd.read_csv('/Users/aishwaryaharpale/tischfield-lab/moseq2_nlp_usages/rST/labels.csv').to_numpy()

In [11]:
## Parameters

K = 1
scoring = 'accuracy'
penalty = 'l2'
num_c = 11
seed = 0

In [12]:
## Pipeline classifier
best_C, best_score, clf = trainer.train_regressor(features, labels, K, scoring, penalty, num_c, seed)

/Users/aishwaryaharpale/opt/anaconda3/envs/moseq2-nlp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [13]:
## Raw ELI5
eli5.show_weights(clf)

In [14]:
## Get group map for the model
custom_groupings = []
model_file = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/rST_model_1000.p'
index_path = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/moseq2-index.role.yaml'
group_map = load_groups(index_path, custom_groupings)

num_transitions = 300
num_syllables = 70

feature_names = get_feature_names(index_path,model_file, group_map)

100%|██████████| 80/80 [00:00<00:00, 97.03it/s] 


In [15]:
## ELI5 with transitions names
eli5.show_weights(clf, feature_names=feature_names)

## gOT

In [14]:
## Get features and labels
features = pd.read_csv('/Users/aishwaryaharpale/tischfield-lab/moseq2_nlp_usages/gOT/features.csv').to_numpy()
labels = pd.read_csv('/Users/aishwaryaharpale/tischfield-lab/moseq2_nlp_usages/gOT/labels.csv').to_numpy()

In [15]:
## Parameters

K = 1
scoring = 'accuracy'
penalty = 'l2'
num_c = 11
seed = 0

In [16]:
## Pipeline classifier
best_C, best_score, clf = trainer.train_regressor(features, labels, K, scoring, penalty, num_c, seed)

ValueError: could not convert string to float: '[0.02164686 0.03777589 0.02631579 0.02419355 0.02419355 0.03056027\n 0.03056027 0.03353141 0.02843803 0.01740238 0.02249576 0.01315789\n 0.02249576 0.01697793 0.01867572 0.02419355 0.03225806 0.0114601\n 0.02758913 0.02419355 0.01612903 0.0229202  0.00764007 0.00891341\n 0.02419355 0.01061121 0.024618   0.01103565 0.01994907 0.01485569\n 0.02249576 0.01103565 0.00594228 0.01400679 0.00891341 0.00933786\n 0.00551783 0.01740238 0.00636672 0.01697793 0.01443124 0.00636672\n 0.02079796 0.01061121 0.01315789 0.01061121 0.00382003 0.00424448\n 0.00466893 0.00466893 0.01061121 0.01018676 0.01103565 0.00679117\n 0.00212224 0.00848896 0.00976231 0.01188455 0.00594228 0.00339559\n 0.0008489  0.00594228 0.00297114 0.00424448 0.00466893 0.00806452\n 0.00891341 0.00551783 0.00254669 0.00594228 0.00976231]'

In [19]:
## Raw ELI5
eli5.show_weights(clf)

In [20]:
## Get group map for the model
custom_groupings = []
model_file = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/gOT_model_1000.p'
index_path = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/moseq2-index.role.yaml'
group_map = load_groups(index_path, custom_groupings)

num_transitions = 300
num_syllables = 70

feature_names = get_feature_names(index_path,model_file, group_map)

100%|██████████| 80/80 [00:01<00:00, 74.12it/s]


In [21]:
## ELI5 with transitions names
eli5.show_weights(clf, feature_names=feature_names)

## gST

In [22]:
## Get features and labels
features = pd.read_csv('/Users/aishwaryaharpale/tischfield-lab/moseq2_nlp_transitions/gST/features.csv').to_numpy()
labels = pd.read_csv('/Users/aishwaryaharpale/tischfield-lab/moseq2_nlp_transitions/gST/labels.csv').to_numpy()

In [23]:
## Parameters

K = 1
scoring = 'accuracy'
penalty = 'l2'
num_c = 11
seed = 0

In [24]:
## Pipeline classifier
best_C, best_score, clf = trainer.train_regressor(features, labels, K, scoring, penalty, num_c, seed)

/Users/aishwaryaharpale/opt/anaconda3/envs/moseq2-nlp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [25]:
## Raw ELI5
eli5.show_weights(clf)

In [26]:
## Get group map for the model
custom_groupings = []
model_file = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/gST_model_1000.p'
index_path = '/Users/aishwaryaharpale/Downloads/2021-02-19_Meloxicam/moseq2-index.role.yaml'
group_map = load_groups(index_path, custom_groupings)

num_transitions = 300
num_syllables = 70

feature_names = get_feature_names(index_path,model_file, group_map)

100%|██████████| 80/80 [00:00<00:00, 84.84it/s]


In [27]:
## ELI5 with transitions names
eli5.show_weights(clf, feature_names=feature_names)